머신러닝을 배울 때는 인공적으로 만들어진 데이터셋이 아닌 실제 데이터로 실험해보는 것이 가장 좋다.
여러 분양 걸쳐 공개된 데이터셋이 아주 많이 있고, 다음은 데이터를 구하기 가장 좋은 곳이다.

- 유명한 공개 데이터 저장소
    - UC 얼바인(Irvine) 머신러닝 저장소(http://archive.ics.uci.edu/ml)
    - 캐글(Kaggle) 데이터셋(http://www.kaggle.com/datasets)
    - 아마존 AWS 데이터셋(https://registry.opendata.aws)

- 메타 포털(공개 데이터 저장소가 나열되어 있다.)
    - 데이터 포털(Data Portals)(http://dataportals.org)
    - 오픈 데이터 모니터(Open Data Monitor)(http://opendatamonitor.eu)
    - 퀀들(Quandl)(http://quandl.com)

- 인기 있는 공개 데이터 저장소가 나열되어 있는 다른 페이지
    - 위키백과 머신러닝 데이터셋 목록(https://en.wikipedia.org/wiki/List_of_datasets_for_machine-learning_research)
    - Quora.com(https://homl.info/10)
    - 데이터셋 서브레딧(subreddit)(http://www.reddit.com/r/datasets)

chapter 2에서는 StatLib 저장소에 있는 캘리포니아 주택 가격(California Housing Prices)데이터 셋을 사용한다.

## Download datasets

맨 처음 할 일은 캘리포니아 인구조사 데이터를 사용해 캘리포니아의 주택 가격 모델을 만드는 것이다.  

이 데이터는 캘리포니아의 블록 그룹마다 인구, 중간 소득, 중간 주택가격 등을 담고 있다.  
블록 그룹은 미국 인구조사국에서 샘플 데이터를 발표하는 데 사용하는 최소한의 지리적 단위. 하나의 블록 그룹은 보통 600 ~ 3000명의 인구를 나타낸다.(여기서는 간단하게 구역이라고 부르겠다.)  

이 데이터로 모델을 학습시켜서 다른 측정 데이터가 주어졌을 때, 구역의 중간 주택 가격을 예측해야 한다.

## 문제 정의 1. 비즈니스의 목적이 정확히 무엇인가?
상사에게 첫 번째로 할 질문은 "비즈니스의 목적이 정확히 무엇인가요?"이다.  
모델 만들기가 최종 목적은 아닐 것이다. 회사에서는 이 모델을 어떻게 사용해 이익을 얻으려고 할까? 목적을 아는 것은 문제를 어떻게 구성할지, 어떤 알고리즘을 선택할지, 모델 평가에 어떤 성능 지표를 사용할지, 모델 튜닝을 위해 얼마나 노력을 투여할지 결정하기 때문에 아주 중요하다.

상사가 이 모델의 출력(구역의 중간 주택 가격에 대한 예측)이 여러 가지 다른 신호와 함께 다른 머신러닝 시스템의 입력으로 사용된다고 이야기한다.  
뒤따르는 시스템이 해당 지역에 투자할 가치가 있는지 결정한다. 이 결정이 수익에 직결되기 때문에 올바르게 예측하는 것은 매우 중요하다.

![1](doc_img/chapter2_1.png)

- 컴포넌트  
    컴포넌트를 한마디로 표현하자면 소프트웨어 시스템에서 독립적인 업무 또는 독립적인 기능을 수행하는 모듈로 이후 시스템을 유지보수 하는데 있어 교체 가능한 부품이다.
    소프트웨어 컴포넌트는 하드웨어의 그래픽 카드와 같은 개념으로 독립적인 기능을 수행하는 소프트웨어 모듈이라고 설명할 수 있으며, 소프트웨어 컴포넌트는 컴포넌트라는 말로 대체되어 사용되고 있다.
    

- 파이프라인  
    데이터 처리 컴포넌트(component)들이 연속되어 있는 것을 데이터 파이프라인(pipeline)이라고 한다.  
    머신러닝 시스템은 데이터를 조작하고 변환할 일이 많아 파이프라인을 사용하는 일이 매우 흔하다.

    보통 컴포넌트들은 비동기적으로 동작한다. 각 컴포넌트는 많은 데이터를 추출해 처리하고 그 결과를 다른 데이터 저장소로 보낸다.  
    그러면 일정 시간 후 파이프라인의 다음 컴포넌트가 그 데이터를 추출해 자신의 출력 결과를 만든다. 각 컴포넌트는 완전히 독립적이다.  
    즉, 컴포넌트 사이의 인터페이스는 데이터 저장소뿐이다. 이는 시스템을 이해하기 쉽게 만들고, 각 팀은 각자의 컴포넌트에 집중할 수 있다.  
    한 컴포넌트가 다운되더라도 하위 컴포넌트는 문제가 생각 컴포넌트의 마지막 출력을 사용해 평상시와 같이 계속 동작할 수 있다. 그래서 시스템이 매우 견고해진다.

개인적인 이해
------------
여기서는 전반적인 시스템 구조를 명시하고자 하는 것 같다.  
예를 들어 A와 B라는 컴포넌트가 있다고 할때, 
 - 각 컴포넌트는 다른 컴포넌트들과는 비동기적이다.
 - A컴포넌트가 처리한 데이터를 저장소에 저장해두면 다음 단계의 컴포넌트, B컴포넌트가 이를 처리하고 결과를 저장소에 저장한다.
 - 즉, 저장소를 제외하면 각 컴포넌트는 다른 컴포넌트에 영향을 주지도, 받지도 않고, 저장소에 있는 데이터를 처리하기만 하는 독립적인 형태를 구성하고 있다는 것.

그림을 통해 이해하자면, 
 - 처음 상위 컴포넌트에 구역 데이터가 입력, 처리되어 구역 가격을 결정하게 된다. 여기서 A라는 컴포넌트가 동작한 것이고, 그 결과가 저장소에 저장되었다.
 - 이후 나의 컴포넌트라고 되어있는 것을 B컴포넌트라 할때, A컴포넌트와는 독립적으로 동작하게 되고, 사실상 A와의 접점은 저장소에 들어있는 A컴포넌트가 처리한 데이터 밖에 없다.


## 문제 정의 2. 현재 솔루션은 어떻게 구성되어 있는가?
다음으로 상사에게 던질 질문은 '현재 솔루션은 어떻게 구성되어 있나요?' 이다.  
현재 상황은 문제 해결 방법에 대한 정보는 물론이고 참고 성능으로도 사용할 수 있다.  
상사가 현재는 구역 주택 가격을 전문가가 수동으로 추정한다고 알려주었다. 한 팀이 구역에 관한 최신 정보를 모으고 있는데, 중간 주택 가격을 얻을 수 없을 때는 복잡한 규칙을 사용하여 추정한다.

이는 비용과 시간이 많이 들고 추정 결과도 썩 좋지 못하다.  
실제 중간 주택 가격을 구해보면 팀에서 추정한 것이 20% 이상 벗어났을 때가 많다. 이런 이유로 회사는 구역의 데이터를 기반으로 중간 주택 가격을 예측하는 모델을 훈련시키는 쪽이 유용하다고 생각한다.  
인구 조사 데이터에는 다른 데이터는 물론 수천 개의 중간 주택 가격을 포함하므로 이 작업에 매우 적합한 데이터셋으로 보인다.

이제 이런 정보들을 가지고 시스템을 설계할 준비가 되었다.  
먼저 문제를 정의해야 한다. 
 - **label된 훈련 샘플**이 있으니 전형적인 **지도 학습 작업**.
 - **값을 예측** 해야하므로 전형적인 **회귀 문제**.
 - **예측에 사용할 특성이 여러 개**이므로 **다중 회귀(multiple regression)**
 - 각 구역마다 **하나의 값을 예측하므로** **단변량 회귀(univariate regression)**, 구역마다 **여러 값을 예측한다면** **다변량 회귀(multivariate regression)**
 - 이 시스템으로 들어오는 데이터에 연속적인 흐름이 없으므로 빠르게 변하는 데이터에 적응하지 않아도 되고, 데이터가 메모리에 들어갈 만큼 충분히 작으므로 일반적인 배치 학습이 적절.

정리
---------
1. 비즈니스의 목적. 즉, 생성한 모델을 통해 얻으려는 것이 무엇인가를 파악해야 한다. 이를 통해 사용하려는 모델, 알고리즘 등 전반적인 사항들을 결정한다.
2. 현재 솔루션은 어떻게 구성되어 있나. 현재 우리가 가진 해결 방안이 어떤 것이 있는가? 이것이 효과적인 방법인가 혹은 그리 좋지 못한 방안일 경우 이를 타개할 방법은 무엇이 있는가?

## 성능 측정 지표 선택
다음 단계는 성능 측정 지표를 선택하는 것이다. 회귀 문제의 전형적인 성능 지표는 **평균 제곱근 오차 <sup>Root Mean Square Error - RMSE</sup>** 이다.  
오차가 커질수록 이 값은 더욱 커지므로 예측에 얼마나 많은 오류가 있는지 가늠하게 해준다.  
![2](doc_img/chapter2_2.png)

 - m은 RMSE를 측정할 데이터셋에 있는 샘플 수. 예로 2000개 구역의 검증 세트에 대해 RMSE를 평가한다면 m = 2,000이다.  
 - $x^{(i)}$ 는 데이터셋에 있는 i번째 샘플(label을 제외한)의 전체 특성값의 벡터이고,  
   $y^{(i)}$ 는 해당 레이블(해당 샘플의 기대 출력값)이다.  
   ex) -118.29°, 위도 33.91°에 위치하고, 중간 소득이 $38,372이며, 주민이 1,416명, 중간 주택 가격이 $156,400라면,  
   ![4](doc_img/chapter2_5.png)
   ![5](doc_img/chapter2_6.png)

 - X는 데이터셋에 있는 모든 샘플의 모든 특성값(label은 제외)을 포함하는 행렬이다.   
 샘플하나가 하나의 행이어서 i번째 행은 $x^{(i)}$의 **전치**와 같고 $(x^{(i)})^T$으로 표기한다.  
 예를 들어 첫 번째 구역이 앞의 예와 같다면 행렬 X는 다음과 같다.  
 ![6](doc_img/chapter2_7.png)

 - h는 시스템의 예측 함수이며 가설(hypothesis)이라고도한다. 시스템이 하나의 샘플 특성 백터 x<sup>(i)</sup>를 받으면  
   그 샘플에 대한 예측값  $y\hat{}^{(i)} = h(x^{(i)})$를 출력한다.  (y hat이라 읽음)
   ![7](doc_img/chapter2_8.png)

    예를 들어 시스템이 첫 번째 구역의 중간 주택 가격을 $\158,400$라고 예측한다면 $\y\hat{}^{(i)} = h(x^{(i)})$ = 158,400이다.
      
    이 구역에 대한 예측 오차는 ![8](doc_img/chapter2_9.png)

 - RMSE(X, h)는 가설 h를 사용하여 일련의 샘플을 평가하는 비용 함수(cost function)이다.

RMSE가 일반적으로 회귀 문제에 선호되는 성능 측정 방법이지만 경우에 따라 다른 함수를 사용할 수도 있다.  
예를 들어 이상치로 보이는 구역이 많다고 가정하면, 평균 절대 오차<sup>Mean Absolute Error</sup>(평균 절대 편차 <sup>Mean Absolute Deviation</sup> 라고도 한다)를 고려해볼 수 있다.  
![img](doc_img/chapter2_10.png)

RMSE와 MAE 모두 예측값의 벡터와 타깃값의 벡터 사이의 거리를 재는 방법이다. 거리 측정에는 여러가지 방법(또는 Norm)이 가능하다.
 - 제곱항을 합한 것의 제곱근(RMSE) 계산은 유클리디안 노름(Euclidean Norm)에 해당한다. $l_{\2}$ 노름이라고도 부르며 다음과 같이 표시한다.
   ![img](doc_img/chapter2_12.png)
 - 절댓값의 합을 계산하는 것은 $l_{\1}$ 노름에 해당하며 다음과 같이 표기한다. ![img](doc_img/chapter2_13.png)  
   이는 도시의 구획이 직각으로 나뉘어 있을 때 이 도시의 두 지점 사이의 거리를 측정하는 것과 같아 맨해튼 노름(Manhattan Norm)이라고도 한다.
 - 일반적으로 원소가 n개인 벡터 v의 $l_{\k}$노름은 다음과 같이 정의한다.
   ![img](doc_img/chapter2_14.png)  
   $l_{\0}$은 단순히 벡터에 있는 0이 아닌 원소의 수이고, $l_{\infty}$는 벡터에서 가장 큰 절댓값이 된다.
 - 노름의 지수가 클수록 큰 값의 원소에 치우치며 그래서 RMSE가 MAE보다 조금 더 이상치에 민감하다. 하지만 이상치가 매우 드물면 RMSE가 잘 맞아 일반적으로 널리 사용된다.

다음 코드를 통해 데이터를 추출하도록 한다.

In [1]:
import os
import tarfile
import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/rickiepark/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

fetch_housing_data()를 호출하면 현재 작업공간에 datasets/housing 디렉터리를 만들고 housing.tgz파일을 내려받고 같은 디렉터리에 압축을 풀어 housing.csv 파일을 만든다.

In [2]:
fetch_housing_data()

이제 Pandas를 사용해 데이터를 읽는다. 데이터를 읽는 간단한 함수도 만들도록한다.

In [3]:
import pandas as pd

def load_housing_data(housing_path = HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

DataFrame의 head()메서드를 사용해 처음 다섯 행을 확인해보자.

In [4]:
housing = load_housing_data()
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


각 행은 하나의 구역을 나타낸다.  
특성은 longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, median_house_value, ocean_proximity등 10개로 구성

info()메서드는 데이터에 대한 간략한 설명과 특히 전체 행 수, 각 특성의 데이터 타입과 널이 아닌 값의 개수를확인하는데 유용하다.

In [5]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
longitude             20640 non-null float64
latitude              20640 non-null float64
housing_median_age    20640 non-null float64
total_rooms           20640 non-null float64
total_bedrooms        20433 non-null float64
population            20640 non-null float64
households            20640 non-null float64
median_income         20640 non-null float64
median_house_value    20640 non-null float64
ocean_proximity       20640 non-null object
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


데이터 셋에 20,640개의 샘플이 들어 있다.